This notebook automatizes the conversion of VHDR files to EDF files. 

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# main_folder = "/content/gdrive/My Drive/epigame-folder/"


In [ ]:
# !pip install mne
# !!pip install EDFlib-Python

import mne

In [ ]:
from datetime import datetime

def format_datetime(raw_brainvision_file):
    # Fetch the start date and time from the raw data's info dictionary
    original_date_time = str(raw_brainvision_file.info['meas_date'])

    parsed_date_time = datetime.strptime(original_date_time, "%Y-%m-%d %H:%M:%S%z")
    iso8601_date_time = parsed_date_time.strftime("%Y-%m-%dT%H:%M:%S")

    print(original_date_time, "converted to", iso8601_date_time)

    raw_brainvision_file.set_meas_date(iso8601_date_time)
    
    return raw_brainvision_file

In [ ]:
import os
import pandas as pd

def find_bad_channels(ieeg_dir):
    """
    For each ieeg file, find bad channels by checking the corresponding channels.tsv file.

    Parameters:
    - ieeg_dir (str): Path to the directory containing ieeg files.

    Returns:
    - bad_channels_dict (dict): Dictionary with ieeg filenames as keys and lists of bad channels as values.
    """
    bad_channels_dict = {}

    for ieeg_file_name in os.listdir(ieeg_dir):
        if ieeg_file_name.endswith(".ieeg"):
            ieeg_file_path = os.path.join(ieeg_dir, ieeg_file_name)

            # Extract sub_id and run_id from the ieeg filename
            sub_id = ieeg_file_name.split('_')[0]
            run_id = ieeg_file_name.split('_')[-2]

            # Search for all channels.tsv files in the channels_tsv_dir
            for channels_tsv_name in os.listdir(ieeg_dir):
                if channels_tsv_name.endswith("_channels.tsv") and sub_id in channels_tsv_name and f"run-{run_id}" in channels_tsv_name:
                    channels_tsv_path = os.path.join(ieeg_dir, channels_tsv_name)

                    # Load channel information from channels.tsv
                    channels_df = pd.read_csv(channels_tsv_path, delimiter='\t')

                    # Extract bad channels
                    bad_channels = channels_df[channels_df['status'] == 'bad']['name'].tolist()

                    # Store bad channels in the dictionary
                    bad_channels_dict[f"{sub_id}_{run_id}"] = bad_channels
                    break  # Break the loop once matching channels_tsv is found
                
    return bad_channels_dict

def exclude_bad_channels(raw, bad_channels):
    """
    Exclude bad channels from the raw data.

    Parameters:
    - raw (mne.io.Raw): MNE Raw object.
    - bad_channels (list): List of bad channel labels.

    Returns:
    - raw (mne.io.Raw): MNE Raw object with bad channels excluded.
    """
    # Exclude bad channels
    raw_copy = raw.copy().drop_channels(bad_channels)
    return raw_copy

In [ ]:
def convert_vhdr_to_edf(vhdr_file, output_dir, bad_channels_dict):
    """
    Converts a BrainVision VHDR file to EDF format using MNE library,
    excluding bad channels found by find_bad_channels function.

    Parameters:
    - vhdr_file (str): Path to the input VHDR file.
    - output_dir (str): Directory to save the output EDF file.
    - bad_channels_dict (dict): Dictionary with ieeg filenames as keys and lists of bad channels as values.
    """
    # Load data using MNE
    raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)
    raw.close()
    # Extract bad channels for the current file
    bad_channels = bad_channels_dict.get(os.path.basename(vhdr_file), [])

    # Exclude bad channels
    raw = exclude_bad_channels(raw, bad_channels)

    raw = format_datetime(raw)
    # Extract subject ID and run number from the filename
    file_name = os.path.basename(vhdr_file)
    subject_id = file_name.split('_')[0]
    run_number = file_name.split('_')[-2]

    # Export to EDF
    edf_file = os.path.join(output_dir, f"{subject_id}_{run_number}.edf")
    mne.export.export_raw(edf_file, raw, fmt='edf', overwrite=True)

    print(f"Converted {vhdr_file} to {edf_file}")


def parse_and_convert_data(root_dir, output_dir):
    """
    Recursively parses the input directory, finds BrainVision VHDR files,
    extracts subject ID and run number, and converts them to EDF format.

    Parameters:
    - root_dir (str): Path to the main directory containing "sub-***" subdirectories.
    - output_dir (str): Directory to save the output EDF files.
    """
    for subject_dir in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject_dir)

        # Check if it's a "sub-***" directory
        if os.path.isdir(subject_path) and subject_dir.startswith("sub-"):
            presurgery_path = os.path.join(subject_path, "ses-presurgery", "ieeg")

            # Check if "ses-presurgery/ieeg" directory exists
            if os.path.exists(presurgery_path) and os.path.isdir(presurgery_path):
                for file in os.listdir(presurgery_path):
                    # Check if the file is a BrainVision VHDR file
                    if file.endswith(".vhdr"):
                        vhdr_file = os.path.join(presurgery_path, file)
                        convert_vhdr_to_edf(vhdr_file, output_dir)

# Replace 'your_input_directory' and 'your_output_directory' with the actual paths
input_directory = main_folder + 'Ext-val/'
output_directory = main_folder + 'Ext-val-edf/'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Find bad channels
bad_channels_result = find_bad_channels(input_directory)

# Iterate through ieeg files and convert to EDF
for ieeg_file_name in os.listdir(input_directory):
    if ieeg_file_name.endswith(".vhdr"):
        ieeg_file_path = os.path.join(input_directory, ieeg_file_name)
        convert_vhdr_to_edf(ieeg_file_path, output_directory, bad_channels_result)
